# Etude des régimes de temps de l'Atlantique nord


**Auteur : FERRY Frédéric (ENM/C3M) - Novembre 2022.
mailto:frederic.ferry@meteo.fr

En météorologie, le concept de **régime de temps** a été introduit il y a environ 70 ans (Rex, 1951). Il est basé sur l'idée que la **circulation atmosphérique de grande échelle des moyennes latitudes** peut être décrite par un **nombre fini d'états atmosphériques** possibles qui se manifestent dans des **schémas d'écoulement de grande échelle quasi-stationnaires, persistants et récurrents**. Étant donné que le temps diffère d'un jour à l'autre et évolue continuellement, classer des situations météorologiques en un nombre fini d'états variant lentement n'est pas une tâche simple. Il existe de nombreuses façons de définir les régimes météorologiques. Si l'on se réfère à la propriété de récurrence (**situations les plus fréquentes au cours d'une période climatologique donnée**), la **classification automatique** ("cluster "analysis") est aujourd'hui l'approche la plus courante pour identifier les régimes de temps. En ce qui concerne la quasi-stationnarité et la persistance, les régimes de temps représentent au sens statistique les **états dans lesquels l'écoulement de grande échelle réside pendant une période prolongée (une semaine à un mois)**.

**Ainsi, le régime de temps, principale entité physique des fluctuations atmosphériques aux moyennes latitudes, fournit une grille de lecture simplificatrice du climat des latitudes moyennes et de son évolution**. Les changements du temps qu'il fait peuvent se comprendre comme le passage d'un régime à un autre. La variabilité climatique peut, quant à elle, s'interpréter comme la conséquence sur une longue période de transitions privilégiées vers un régime donné.

**On s'intéressera ici à l'étude des régimes de temps de l'Atlantique Nord à partir de séries quotidiennes de géopotentiel à 500 hPa et de pression réduite au niveau de la mer**. 

- Etape 1 : ouverture et traitement des données quotidiennes
- Etape 2 : étude des données quotidiennes sur un mois donné
- Etape 3 : décomposition des données en temps et en espace (analyse en composantes principales, ACP)
- Etape 4 : classification en 4 classes = régimes (méthode k-means)
- Etape 5 : obtention des régimes de temps
- Etape 6 : temps sensible associé aux régimes de temps (étape à faire uniquement pour les régimes de temps d'hiver sur l'Atlantique nord)
- Etape 7 : étude des occurrences saisonnières des régimes de temps
- Etape 8 : corrélation régimes/indices océaniques (Nino 3.4, TNA, AMV)
- Etape 9 : retour sur une saison particulière

**Remarque : la technique d'analyse en composantes principales (ACP) et la méthode de classification de type k-means seront abordés en deuxième année. Ces outils d'analyse de données seront utilisés ici comme des "boîtes noires"**.

Concepts Python illustrés :

- Exploitation de fichiers de données météorologiques au format netcdf (xarray/netCDF4)
- Calcul d'anomalies quotidiennes (méthode groupby de xarray)
- Création de séries temporelles (pandas)
- Tracé de cartes (matplotlib/cartopy)
- Régression linéaire (module LinearRegression de sklearn, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- Analyse en composantes principales (package eofs, https://ajdawson.github.io/eofs/latest/)
- Représentation des données dans un espace des phases (scatterplot 2D ou 3D)
- Tracé de densité de points (module gaussian_kde de scipy)
- Classification K-means (module KMeans de sklearn, https://scikit-learn.org/stable/modules/clustering.html)
- Réalisation de cartes composites (moyenne par classe)

<div class="alert alert-warning">
<b>Instructions : </b>
<p><b>1) </b>Exécuter les cellules qui suivent de façon séquentielle</p>
<p><b>2) </b>Répondre aux questions (celulles de couleur jaune) dans les cadres réponses dédiés (cellules de couleur verte)</p>
<p><b>3) </b>Sauvegarder le calepin final au format pdf</p>
</div>

<div class="alert alert-danger">
<b>Attention : l'obtention des régimes de temps nécessite d'utiliser des méthodes statistiques qui seront étudiées en deuxième année (analyse en composantes principales - étape 3, classification automatique - étape 4). On utilisera ici ces méthodes comme des boîtes noires. </b>
</div>

In [ ]:
%matplotlib inline

import calendar
from calendar import isleap
import datetime
import os

import numpy as np
from scipy.stats import gaussian_kde

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import xarray as xr
from netCDF4 import Dataset

import pandas as pd
from pandas import Series
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.path as mpath

from cartopy import config
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from eofs.standard import Eof

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dir_data='./data/'
dir_res='./result/'
dir_figs='./figs/'
dir_anim='./anim/'

if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)
if not os.path.exists(dir_res):
    os.makedirs(dir_res)

# Etape 1 : ouverture et traitement des données quotidiennes

In [ ]:
var_text=input("Z500 ou Pmer : Z500/MSLP? ")

if var_text=='Z500':
    infile = dir_data+'era5_z500_natl_daily_1950-2020_1deg5.nc'
    varname='z'
    var_div=9.81
    units='m'

if var_text=='MSLP':
    infile = dir_data+'era5_msl_natl_daily_1950-2020_1deg5.nc'
    varname='msl'
    var_div=100
    units='hPa'

data    = xr.open_dataset(infile)
print(data.variables)

In [ ]:
lat  = data.lat.values
lon  = data.lon.values

domain='North Atlantic'
domain_name='natl'
latS=lat[0]
latN=lat[-1]
lonW=lon[0]
lonE=lon[-1]

In [ ]:
season_name=input("Saison hiver ou été : winter/summer ? ")

if season_name=='winter':
    season='Winter'
    if var_text=='MSLP':
        startday  = '1957-12-01T09'
        endday  = '2020-03-31T09'
    if var_text=='Z500':
        startday  = '1957-12-01T18'
        endday  = '2020-03-31T18'
if season_name=='summer':
    season='Summer'
    if var_text=='MSLP':
        startday  = '1958-06-01T09'
        endday  = '2020-09-30T09'
    if var_text=='Z500':
        startday  = '1958-06-01T18'
        endday  = '2020-09-30T18'

# Date index from startday to endday
dates = pd.date_range(startday, endday, freq='D')
print(dates)

# Remove 29/02
#def is_leap_and_29Feb(s):
#    return (s.year % 4 == 0) & ((s.year % 100 != 0) | (s.year % 400 == 0)) & (s.month == 2) & (s.day == 29)
#mask = is_leap_and_29Feb(dates)
#dates=dates[~mask]
#print(dates)

# Select Season
if season_name == 'winter':
    months=np.any([dates.month==12,dates.month==1,dates.month==2,dates.month==3],axis=0)
if season_name == 'summer':
    months=np.any([dates.month==6,dates.month==7,dates.month==8,dates.month==9],axis=0)

dates2=dates[months]
print(dates2)

In [ ]:
data_season = data.sel(time=dates2)
print(data_season)
time  = data_season.time.values

if season_name == 'winter':
    season1= str(data_season.time.values[130])[0:4]
    season2= str(data_season.time.values[-1])[0:4]

if season_name == 'summer':
    season1= str(data_season.time.values[0])[0:4]
    season2= str(data_season.time.values[-1])[0:4]

print(' ----- Saving new seasonnal file from '+startday+ ' to '+endday+ ' on new domain  ----- ')
infile1 = dir_res+varname+'_'+startday+'_'+endday+'_'+domain_name+'.nc'
data_season.to_netcdf(infile1)
print(' new daily seasonnal file over subdomain is here : ')
print(infile1)

In [ ]:
with np.printoptions(threshold=np.inf):
    print(time)

In [ ]:
print(' ----- Computing daily anomalies ----- ')
data_anom = data_season.groupby('time.dayofyear') - data_season.groupby('time.dayofyear').mean('time')
print(data_anom)
print(' ----- Writing netcdf ----- ')
infile2 = dir_res+varname+'_anom_'+startday+'_'+endday+'_'+domain_name+'.nc'
data_anom.to_netcdf(infile2)
print(' netcdf file of daily anomalies is here : ')
print(infile2)

# Etape 2 : étude des données quotidiennes sur un mois donné

In [ ]:
if season_name == 'winter':
    date1='20191201'
    date2='20201230'
    
if season_name == 'summer':
    date1='20030801'
    date2='20030830'
    
data_month    = data_season.sel(time=slice(date1,date2))
data_month_anom    = data_anom.sel(time=slice(date1,date2))

z=data_month[varname]/var_div
z_anom=data_month_anom[varname]/var_div

time  = data_month.time.values

time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]

for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:10]
    
print(data_month)

In [ ]:
projection = ccrs.Orthographic(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
bounds = [(lonW, lonE, latS, latN)]

def make_boundary_path(lon,lat):
    lons,lats=np.meshgrid(lon,lat)
    boundary_path = np.array([lons[-1,:],lats[-1,:]])
    boundary_path = np.append(boundary_path,np.array([lons[::-1,-1],lats[::-1,-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[1,::-1],lats[1,::-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[:,1],lats[:,1]]),axis=1)
    boundary_path = mpath.Path(np.swapaxes(boundary_path, 0, 1))
    return boundary_path

In [ ]:
if varname=='z':
    levels1 = np.arange(4800,6200,100)
    plt_title1 = 'Geopotential height ('+units+') at 500 hPa : '+ str(date1)+'-'+str(date2)
    levels2 = np.arange(-500,550,50)
    plt_title2 = 'Geopotential height anomaly ('+units+') at 500 hPa : '+ str(date1)+'-'+str(date2)

if varname=='msl':
    levels1 = np.arange(980,1040,5)
    plt_title1 = 'Mean Sea Level Pressure ('+units+') : '+ str(date1)+'-'+str(date2)
    levels2 = np.arange(-25,27.5,2.5)
    plt_title2 = 'Mean Sea Level Pressure anomaly ('+units+') : '+ str(date1)+'-'+str(date2)

cmap1='jet'
cmap2='RdBu_r'

In [ ]:
axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(17,10))
fig.suptitle(plt_title1, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(5, 6),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.set_title(date_str[i], fontsize=10)
    p1 = ax.contourf(lon, lat, z[i,:,:], levels1, transform=ccrs.PlateCarree(), cmap=cmap1, extend='both')
    p2 = ax.contour(lon, lat, z[i,:,:], levels1, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)
   
plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_'+str(date1)+'-'+str(date2)
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(17,10))
fig.suptitle(plt_title2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(5, 6),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.set_title(date_str[i], fontsize=10)
    p1 = ax.contourf(lon, lat, z_anom[i,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname=dir_figs+varname+'_anom_'+domain_name+'_'+season_name+'_'+str(date1)+'-'+str(date2)
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(8., 8.))
    fig.suptitle(plt_title2, fontsize=16)
    ax = fig.add_subplot(1,1,1, projection=projection)
    ax.set_title(date_str[i], loc='center')
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    p1 = ax.contourf(lon, lat, z_anom[i,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    cb = fig.colorbar(p1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('Anomaly ('+units+')', fontsize=12)

    figname=dir_anim+varname+'_anom_'+domain_name+'_'+season_name+'_'+date_str[i]
    fig.savefig(figname+'.png', bbox_inches='tight')
    plt.close()
print('png files are in the animation folder, ready to make the animation')

In [ ]:
def make_animation():
    nbimages=len(time)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 1000 ms
    standard_duration = 1000 #show all other frames for 1000 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format(dir_anim)))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir(dir_anim)
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

In [ ]:
gif_filepath = dir_anim+varname+'_anom_'+domain_name+'_'+season_name+'_'+str(date1)+'-'+str(date2)+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Pour la séquence mensuelle choisie, peut-on déjà regrouper subjectivement des situations qui se ressemblent ? Combien de groupes pourrait-on faire ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
<p>
</p>
</div>

# Etape 3 : analyse en composantes principales (ACP)

<div class="alert alert-danger">
<b>Exécuter l'ensemble des cellules de l'étape 3 ci-dessous pour réduire la dimension du problème en décomposant les données en temps et en espace via la méthode d'analyse en composantes principales (ACP)</b>
</div>

In [ ]:
filename = infile1
ncin = Dataset(filename, 'r')
lons = ncin.variables['lon'][:]
lats = ncin.variables['lat'][:]
var = ncin.variables[varname][:]/var_div
ncin.close()

filename = infile2
ncin = Dataset(filename, 'r')
lons = ncin.variables['lon'][:]
lats = ncin.variables['lat'][:]
var_anom = ncin.variables[varname][:]/var_div
ncin.close()

In [ ]:
wgts = np.sqrt(np.cos(np.deg2rad(lats)))[:, np.newaxis]
solver = Eof(var_anom, weights=wgts, center=True)

In [ ]:
n=15
eofs = solver.eofs(neofs=n, eofscaling=1)
pcs = solver.pcs(npcs=n, pcscaling=0)
#pcs_norm = solver.pcs(npcs=n, pcscaling=1)
varfrac = solver.varianceFraction()
print("% de variance expliquée par les 10 premiers EOFs : ", 100*varfrac[0:10].sum())

In [ ]:
axes_class = (GeoAxes, dict(map_projection=projection))

fig = plt.figure(figsize=(15,6))
fig.suptitle('EOFs : '+var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(3, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='each', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.set_title('EOF'+str(i+1)+' ('+str(int(varfrac[i]*100))+'%)', fontsize=10, loc='center')
    cf = ax.contourf(lons, lats, eofs[i]*1e5, transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
    c = ax.contour(lons, lats, eofs[i]*1e5, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_eofs'
fig.savefig(figname+'.png', bbox_inches='tight')

fig = plt.figure(figsize=(15,6))
fig.suptitle('PCs : '+var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)

plt.subplots_adjust(hspace=0.5, wspace=0.4)

for i in range(0, 15):
        plt.subplot(3, 5, i+1)
        plt.title('PC'+str(i+1)+'(t)')
        plt.axhline(0, color='k', linewidth=0.5)
        if varname=='z':
            plt.ylim(-9000, 9000)
        if varname=='msl':
            plt.ylim(-400, 400)
        plt.plot(pcs[:,i], color='k', linewidth=0.5, alpha=0.7)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pcs'
fig.savefig(figname+'.png', bbox_inches='tight')
             
fig = plt.figure(figsize=(15, 8))
fig.suptitle('Variance fraction : '+var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)

eof_num = range(1, 16)
plt.bar(eof_num, varfrac[0:15], width=0.5)
plt.axhline(0, color='k')
plt.xticks(range(1, 16))
plt.xlabel('EOF #')
plt.ylabel('Variance Fraction')
plt.xlim(1, 15)
plt.ylim(np.min(varfrac), np.max(varfrac)+0.01)
plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_varfrac'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
eof1=eofs[0]*(-1)
eof2=eofs[1]
eof3=eofs[2]

pc1=pcs[:,0]*(-1)
pc2=pcs[:,1]
pc3=pcs[:,2]

def plot_pc(ax):
    plt.xlabel('Time (days)')
    plt.axhline(0, color='k')
    if varname=='z':
        ax.set_ylim(-9000, 9000)
    if varname=='msl':
        ax.set_ylim(-400, 400)
    return ax

if varname=='z':
    clevs = np.linspace(-3, 3, 11)
if varname=='msl':
    clevs = np.linspace(-5, 5, 11)

fig = plt.figure(figsize=(15, 15))

fig.suptitle('EOFs and PCs : '+var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)
ax = fig.add_subplot(3, 2, 1, projection=projection)
plt.title('EOF1 ('+str(int(varfrac[0]*100))+'%)', fontsize=10, loc='center')
ax.coastlines()
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, eof1*1e5, clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, eof1*1e5, clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=70, shrink=0.6, pad=0.1, extendrect='True')
cb.set_label('$10^{5}$', fontsize=12)

ax = fig.add_subplot(3, 2, 2)
plot_pc(ax)
plt.title('PC1')
plt.plot(pc1, color='k', linewidth=1)

ax = fig.add_subplot(3, 2, 3, projection=projection)
plt.title('EOF2 ('+str(int(varfrac[1]*100))+'%)', fontsize=10, loc='center')
ax.coastlines()
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, eof2*1e5, clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, eof2*1e5, clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=70, shrink=0.6, pad=0.1, extendrect='True')
cb.set_label('$10^{5}$', fontsize=12)

ax = fig.add_subplot(3, 2, 4)
plot_pc(ax)
plt.title('PC2')
plt.plot(pc2, color='k', linewidth=1)

ax = fig.add_subplot(3, 2, 5, projection=projection)
plt.title('EOF3 ('+str(int(varfrac[2]*100))+'%)', fontsize=10, loc='center')
ax.coastlines()
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, eof3*1e5, clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, eof3*1e5, clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=70, shrink=0.6, pad=0.1, extendrect='True')
cb.set_label('$10^{5}$', fontsize=12)

ax = fig.add_subplot(3, 2, 6)
plot_pc(ax)
plt.title('PC3')
plt.plot(pc3, color='k', linewidth=1)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_eofs_pcs'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
day=int(input("Entrer un numéro de jour pour la reconstruction du champ : "))
field=var_anom[day]

#5 EOFS
reconst=eofs[0]*pcs[day,0]*np.std(
    pcs[:,0])+eofs[1]*pcs[day,1]*np.std(
    pcs[:,1])+eofs[2]*pcs[day,2]*np.std(
    pcs[:,2])+eofs[3]*pcs[day,3]*np.std(
    pcs[:,3])+eofs[4]*pcs[day,4]*np.std(
    pcs[:,4])

reconst=reconst/wgts

#15 EOFS
reconst2=eofs[0]*pcs[day,0]*np.std(
    pcs[:,0])+eofs[1]*pcs[day,1]*np.std(
    pcs[:,1])+eofs[2]*pcs[day,2]*np.std(
    pcs[:,2])+eofs[3]*pcs[day,3]*np.std(
    pcs[:,3])+eofs[4]*pcs[day,4]*np.std(
    pcs[:,4])+eofs[5]*pcs[day,5]*np.std(    
    pcs[:,5])+eofs[6]*pcs[day,6]*np.std(    
    pcs[:,6])+eofs[7]*pcs[day,7]*np.std(
    pcs[:,7])+eofs[8]*pcs[day,8]*np.std(
    pcs[:,8])+eofs[9]*pcs[day,9]*np.std(
    pcs[:,9])+eofs[10]*pcs[day,10]*np.std(
    pcs[:,10])+eofs[11]*pcs[day,11]*np.std(
    pcs[:,11])+eofs[12]*pcs[day,12]*np.std(
    pcs[:,12])+eofs[13]*pcs[day,13]*np.std(
    pcs[:,13])+eofs[14]*pcs[day,14]*np.std(
    pcs[:,14])

reconst2=reconst2/wgts

fig = plt.figure(figsize=(15, 5))

fig.suptitle('Reconstruction from the EOFs and PCs', fontsize=16)
ax = fig.add_subplot(131, projection=projection)
plt.title('Anomaly field', fontsize=10, loc='center')
ax.coastlines()
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, field, levels2, cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, field, levels2, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label(units, fontsize=12)

ax = fig.add_subplot(132, projection=projection)
plt.title('Reconstruction with 5 EOFs', fontsize=10, loc='center')
ax.coastlines()
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, reconst, levels2, cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, reconst, levels2, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label(units, fontsize=12)

ax = fig.add_subplot(133, projection=projection)
plt.title('Reconstruction with 15 EOFs', fontsize=10, loc='center')
ax.coastlines()
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, reconst2, levels2, cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, reconst2, levels2, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label(units, fontsize=12)

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Pour le jour choisi, la reconstruction du champ sur la base des 15 premiers EOFs est-elle satisfaisante ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
<p>
</p>
</div>

In [ ]:
def plot_phase_space2d(ax):
    plt.title('')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    if varname=='z':
        plt.xlim(-9000, 9000)
        plt.ylim(-9000, 9000)
    if varname=='msl':
        plt.xlim(-200, 200)
        plt.ylim(-200, 200)
    plt.axvline(0, color='k', linestyle='--')
    plt.axhline(0, color='k', linestyle='--')
    return ax

def plot_phase_space3d(ax):
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('PC1 and PC2 phase space density', fontsize=14)
plot_phase_space2d(ax)
xy = np.vstack([pc1,pc2])
z = gaussian_kde(xy)(xy)
#ax.scatter(pc1,pc2, s=10, color = 'r')
ax.scatter(pc1, pc2, cmap='jet', c=z, s=10)
plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pc12_density'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.suptitle('PC1 PC2 and PC3 phase space', fontsize=14)
ax = fig.add_subplot(111, projection='3d')
plot_phase_space3d(ax)
xyz = np.vstack([pc1,pc2,pc3])
z = gaussian_kde(xyz)(xyz)
#ax.scatter(pc1,pc2,pc3, s=10, color = 'r')
ax.scatter(pc1, pc2, pc3, cmap='jet', c=z, s=10)
plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pc123_density'
fig.savefig(figname+'.png', bbox_inches='tight')

# Etape 4 : classification en 4 classes = régimes (méthode k-means)

<div class="alert alert-danger">
<b>Exécuter l'ensemble des cellules de l'étape 4 ci-dessous pour réaliser la classification automatique en 4 régimes via la méthode k-means.</b>
</div>

In [ ]:
pcs = np.array([pc1,pc2,pc3,pcs[:,3],
                pcs[:,4],pcs[:,5],pcs[:,6],
                pcs[:,7],pcs[:,8],pcs[:,9],
                pcs[:,10],pcs[:,11],pcs[:,12],
                pcs[:,13],pcs[:,14]],)
pcs=pcs.transpose()

#Number of time the k-means algorithm will be run with different centroid seeds
n_init=100
#Maximum number of iterations of the k-means algorithm for a single run
max_iter=500

# Number of clusters
kmeans = KMeans(n_clusters=4, n_init=n_init, max_iter=max_iter, algorithm="full", verbose=1)
# Fitting the input data
kmeans = kmeans.fit(pcs)
# Getting the cluster labels
cluster = kmeans.predict(pcs)
# Centroid values
centroids = kmeans.cluster_centers_

print(cluster)
print(cluster.shape)
print(centroids.shape)

In [ ]:
nc1=list(cluster[:]).count(0)
nc2=list(cluster[:]).count(1)
nc3=list(cluster[:]).count(2)
nc4=list(cluster[:]).count(3)

f1=round(nc1/cluster.shape[0]*100,2)
f2=round(nc2/cluster.shape[0]*100,2)
f3=round(nc3/cluster.shape[0]*100,2)
f4=round(nc4/cluster.shape[0]*100,2)

In [ ]:
colors=[""]*len(cluster)
couleur=["peru","gold","grey","turquoise"]
#couleur=["blue","red","green","orange"]
for i in range(len(cluster)):
    colors[i]=couleur[cluster[i]]

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
fig.suptitle('PC1 and PC2 phase space (4 clusters)', fontsize=14)
plot_phase_space2d(ax)
plt.scatter(pc1, pc2, c=colors, s=10)
plt.scatter(centroids[:, 0], centroids[:, 1], c=couleur, s=2000, alpha=1, marker='+');

patch1 = mpatches.Patch(color=couleur[0], label='Cluster 1 : '+str(nc1)+' ('+str(f1)+'%)')
patch2 = mpatches.Patch(color=couleur[1], label='Cluster 2 : '+str(nc2)+' ('+str(f2)+'%)')
patch3 = mpatches.Patch(color=couleur[2], label='Cluster 3 : '+str(nc3)+' ('+str(f3)+'%)')
patch4 = mpatches.Patch(color=couleur[3], label='Cluster 4 : '+str(nc4)+' ('+str(f4)+'%)')
all_handles = (patch1, patch2, patch3, patch4)
leg = ax.legend(handles=all_handles, loc='upper right')

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pc12_clustering'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.suptitle('PC1 PC2 and PC3 phase space (4 clusters)', fontsize=14)
ax = fig.add_subplot(111, projection='3d')
plot_phase_space3d(ax)
ax.scatter(pc1, pc2, pc3, c=colors, s=10)
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], c=couleur, s=2000, alpha=1, marker='+')

patch1 = mpatches.Patch(color=couleur[0], label='Cluster 1 : '+str(nc1)+' ('+str(f1)+'%)')
patch2 = mpatches.Patch(color=couleur[1], label='Cluster 2 : '+str(nc2)+' ('+str(f2)+'%)')
patch3 = mpatches.Patch(color=couleur[2], label='Cluster 3 : '+str(nc3)+' ('+str(f3)+'%)')
patch4 = mpatches.Patch(color=couleur[3], label='Cluster 4 : '+str(nc4)+' ('+str(f4)+'%)')
all_handles = (patch1, patch2, patch3, patch4)
leg = ax.legend(handles=all_handles, loc='upper right')

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pc123_clustering'
fig.savefig(figname+'.png', bbox_inches='tight')

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Comparer les classes (clusters) avec celles du voisin. Que constate t-on ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
<p>
</p>
</div>

# Etape 5 : obtention des régimes de temps

In [ ]:
id_cluster1=np.any([cluster==0],axis=0)
id_cluster2=np.any([cluster==1],axis=0)
id_cluster3=np.any([cluster==2],axis=0)
id_cluster4=np.any([cluster==3],axis=0)

print(id_cluster1)
print(id_cluster2)
print(id_cluster3)
print(id_cluster4)

mean_c1 = var[id_cluster1,:,:].mean(axis=0)
mean_c2 = var[id_cluster2,:,:].mean(axis=0)
mean_c3 = var[id_cluster3,:,:].mean(axis=0)
mean_c4 = var[id_cluster4,:,:].mean(axis=0)

mean_c1_anom = var_anom[id_cluster1,:,:].mean(axis=0)
mean_c2_anom = var_anom[id_cluster2,:,:].mean(axis=0)
mean_c3_anom = var_anom[id_cluster3,:,:].mean(axis=0)
mean_c4_anom = var_anom[id_cluster4,:,:].mean(axis=0)

In [ ]:
if varname=='z':
    clevs = np.linspace(5000, 6000, 11)
    clevs_anom = np.linspace(-150, 150, 11)
if varname=='msl':
    clevs = np.linspace(990, 1020, 15)
    clevs_anom = np.linspace(-8, 8, 17)

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Weather regimes : '+var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)

ax = fig.add_subplot(2, 2, 1, projection=projection)
plt.title('Regime 1 (freq = '+str(f1)+'%)', fontsize=10, loc='center', color=couleur[0])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c1_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c1, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

ax = fig.add_subplot(2, 2, 2, projection=projection)
plt.title('Regime 2 (freq = '+str(f2)+'%)', fontsize=10, loc='center', color=couleur[1])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c2_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c2, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

ax = fig.add_subplot(2, 2, 3, projection=projection)
plt.title('Regime 3 (freq = '+str(f3)+'%)', fontsize=10, loc='center', color=couleur[2])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c3_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c3, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

ax = fig.add_subplot(2, 2, 4, projection=projection)
plt.title('Regime 4 (freq = '+str(f4)+'%)', fontsize=10, loc='center', color=couleur[3])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c4_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c4, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_'+'regimes'
fig.savefig(figname+'.png', bbox_inches='tight')

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Nommer chacun des 4 régimes de temps
</p>
</div>

<div class="alert alert-success">
<b>Réponse en éxécutant la cellule de code ci-dessous </b>
<p>
</p>
</div>

In [ ]:
if domain_name=='natl' and season_name=='winter':
    r1=input("Parmi les propositions suivantes nommer le régime 1 : NAO+ / NAO- / Dorsale Atl / Blocage : ")
    r2=input("Parmi les propositions suivantes nommer le régime 2 : NAO+ / NAO- / Dorsale Atl / Blocage : ")
    r3=input("Parmi les propositions suivantes nommer le régime 3 : NAO+ / NAO- / Dorsale Atl / Blocage : ")
    r4=input("Parmi les propositions suivantes nommer le régime 4 : NAO+ / NAO- / Dorsale Atl / Blocage : ")

if domain_name=='natl' and season_name=='summer':
    r1=input("Parmi les propositions suivantes nommer le régime 1 : Zonal / Anticyclone Gro / Blocage été / Thalweg Atl : ")
    r2=input("Parmi les propositions suivantes nommer le régime 2 : Zonal / Anticyclone Gro / Blocage été / Thalweg Atl : ")
    r3=input("Parmi les propositions suivantes nommer le régime 3 : Zonal / Anticyclone Gro / Blocage été / Thalweg Atl : ")
    r4=input("Parmi les propositions suivantes nommer le régime 4 : Zonal / Anticyclone Gro / Blocage été / Thalweg Atl : ")

regime=[r1, r2, r3, r4]

<div class="alert alert-warning">
<b>Question : Modifier le tableau "couleur" associé au tableau "regime" pour avoir la correspondance suivante : </b>
<p>
<b>NAO+ : rouge</b>
<br>
<b>NAO- : vert</b>
<br>
<b>Dorsale Atlantique : orange</b>
<br>
<b> Blocage Scandinave: bleu</b>
</p>
</div>

<div class="alert alert-success">
<b>Réponse en modifiant la cellule de code ci-dessous </b>
<p>
</p>
</div>

In [ ]:
print(regime)
couleur=["blue","orange","red","green"]
print(couleur)

In [ ]:
colors=[""]*len(cluster)
for i in range(len(cluster)):
    colors[i]=couleur[cluster[i]]

In [ ]:
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Weather regimes : '+var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)

ax = fig.add_subplot(2, 2, 1, projection=projection)
plt.title(r1+' (freq = '+str(f1)+'%)', fontsize=10, loc='center', color=couleur[0])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c1_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c1, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

ax = fig.add_subplot(2, 2, 2, projection=projection)
plt.title(r2+' (freq = '+str(f2)+'%)', fontsize=10, loc='center', color=couleur[1])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c2_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c2, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

ax = fig.add_subplot(2, 2, 3, projection=projection)
plt.title(r3+' (freq = '+str(f3)+'%)', fontsize=10, loc='center', color=couleur[2])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c3_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c3, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

ax = fig.add_subplot(2, 2, 4, projection=projection)
plt.title(r4' (freq = '+str(f4)+'%)', fontsize=10, loc='center', color=couleur[3])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lons, lats)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lons, lats, mean_c4_anom, levels=clevs_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lons, lats, mean_c4, levels=clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly ('+units+')', fontsize=12)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_'+'regimes2'
fig.savefig(figname+'.png', bbox_inches='tight')

# Etape 6 : temps sensible associé aux régimes de temps

In [ ]:
data_t0    = xr.open_dataset(dir_data+'t2m_eur_19790101_20191231.nc')
data_tp0    = xr.open_dataset(dir_data+'tp_eur_19790101_20191231.nc')
print(data_t0)
print(data_tp0)

In [ ]:
if season_name == 'winter':
    d1='1979-12-01T09'
    d2='2018-03-31T09'

if season_name == 'summer':
    d1='1979-06-01T09'
    d2='2018-09-30T09'

    # Date index from startday to endday
d = pd.date_range(d1, d2, freq='D')
print(d)

# Remove 29/02
#mask = is_leap_and_29Feb(d)
#d=d[~mask]
#print(d)

# Select Season
if season_name == 'winter':
    mm=np.any([d.month==12,d.month==1,d.month==2,d.month==3],axis=0)
if season_name == 'summer':
    mm=np.any([d.month==6,d.month==7,d.month==8,d.month==9],axis=0)

dd2=d[mm]
print(dd2)

data_t    = xr.open_dataset(dir_data+'t2m_eur_19790101_20191231.nc').sel(time=slice(d1,d2))
data_t    = data_t.sel(time=dd2)
data_tp    = xr.open_dataset(dir_data+'tp_eur_19790101_20191231.nc').sel(time=slice(d1,d2))
data_tp    = data_tp.sel(time=dd2)


print(data_t0)
print(data_tp0)
print(data_t)
print(data_tp)

lat_t  = data_t.latitude.values
lon_t  = data_t.longitude.values
time_t  = data_t.time.values

In [ ]:
print(' ----- Computing daily anomalies of T2m ----- ')
data_t_anom = data_t.groupby('time.dayofyear') - data_t.groupby('time.dayofyear').mean('time')
print(' ----- Computing daily anomalies of precipitation ----- ')
data_tp_anom = data_tp.groupby('time.dayofyear') - data_tp.groupby('time.dayofyear').mean('time')

t2m_anom=data_t_anom['t2m']
tp_anom=data_tp_anom['tp']*1000

print(t2m_anom.shape)

In [ ]:
print(cluster.shape)
print(dates2)

if season_name=='winter':
    if var_text=='Z500':
        dd1='1979-12-01T18'
        dd2='2018-03-31T18'
    if var_text=='MSLP':
        dd1='1979-12-01T09'
        dd2='2018-03-31T09'
        
if season_name=='summer':
    if var_text=='Z500':
        dd1='1979-06-01T18'
        dd2='2018-09-30T18'
    if var_text=='MSLP':
        dd1='1979-06-01T09'
        dd2='2018-09-30T09'
        
index_y=np.all([dates2>=dd1, dates2<=dd2], axis=0)
cluster_t=cluster[index_y]
print(cluster_t.shape)

id_cluster1_t=np.any([cluster_t==0],axis=0)
id_cluster2_t=np.any([cluster_t==1],axis=0)
id_cluster3_t=np.any([cluster_t==2],axis=0)
id_cluster4_t=np.any([cluster_t==3],axis=0)

mean_c1_anom_t = t2m_anom[id_cluster1_t,:,:].mean(axis=0)
mean_c2_anom_t = t2m_anom[id_cluster2_t,:,:].mean(axis=0)
mean_c3_anom_t = t2m_anom[id_cluster3_t,:,:].mean(axis=0)
mean_c4_anom_t = t2m_anom[id_cluster4_t,:,:].mean(axis=0)

mean_c1_anom_tp = tp_anom[id_cluster1_t,:,:].mean(axis=0)
mean_c2_anom_tp = tp_anom[id_cluster2_t,:,:].mean(axis=0)
mean_c3_anom_tp = tp_anom[id_cluster3_t,:,:].mean(axis=0)
mean_c4_anom_tp = tp_anom[id_cluster4_t,:,:].mean(axis=0)

In [ ]:
#projection2=ccrs.EuroPP()
#projection2=ccrs.PlateCarree()
projection2=ccrs.NearsidePerspective(central_longitude=5.0, central_latitude=55.0)
bounds = [(-20, 30, 30, 80)]

levs_t_anom = np.linspace(-4, 4, 21)

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Composites of 2-meter temperature anomalies : ERA5 DJFM 1979-2018', fontsize=16)

ax = fig.add_subplot(2, 2, 1, projection=projection2)
plt.title(regime[0], fontsize=10, loc='center', color=couleur[0])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c1_anom_t, levels=levs_t_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (°C)', fontsize=12)

ax = fig.add_subplot(2, 2, 2, projection=projection2)
plt.title(regime[1], fontsize=10, loc='center', color=couleur[1])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c2_anom_t, levels=levs_t_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (°C)', fontsize=12)

ax = fig.add_subplot(2, 2, 3, projection=projection2)
plt.title(regime[2], fontsize=10, loc='center', color=couleur[2])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c3_anom_t, levels=levs_t_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (°C)', fontsize=12)

ax = fig.add_subplot(2, 2, 4, projection=projection2)
plt.title(regime[3], fontsize=10, loc='center', color=couleur[3])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c4_anom_t, levels=levs_t_anom, 
                 cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (°C)', fontsize=12)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_t2m_composite_DJFM'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
cmap3='BrBG'

levs_tp_anom = np.linspace(-1, 1, 21)

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Composites of precipitation anomalies : ERA5 1979-2018', fontsize=16)

ax = fig.add_subplot(2, 2, 1, projection=projection2)
plt.title(regime[0], fontsize=10, loc='center', color=couleur[0])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c1_anom_tp, levels=levs_tp_anom, 
                 cmap=cmap3, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (mm/day)', fontsize=12)

ax = fig.add_subplot(2, 2, 2, projection=projection2)
plt.title(regime[1], fontsize=10, loc='center', color=couleur[1])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c2_anom_tp, levels=levs_tp_anom, 
                 cmap=cmap3, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (mm/day)', fontsize=12)

ax = fig.add_subplot(2, 2, 3, projection=projection2)
plt.title(regime[2], fontsize=10, loc='center', color=couleur[2])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c3_anom_tp, levels=levs_tp_anom, 
                 cmap=cmap3, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (mm/day)', fontsize=12)

ax = fig.add_subplot(2, 2, 4, projection=projection2)
plt.title(regime[3], fontsize=10, loc='center', color=couleur[3])
ax.coastlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon_t, lat_t)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
cf = ax.contourf(lon_t, lat_t, mean_c4_anom_tp, levels=levs_tp_anom, 
                 cmap=cmap3, extend='both', transform=ccrs.PlateCarree())
ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('Anomaly (mm/day)', fontsize=12)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_precip_composite_DJFM'
fig.savefig(figname+'.png', bbox_inches='tight')

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Préciser l’impact de chaque régime en temps sensible (température/précipitation) sur l'Europe de l'ouest. Ces impacts sont-ils spatialement homogènes pour tous les régimes ? Sinon, préciser les disparités régionales. Comparer éventuellement aux composites suivants :
    
http://blogs.reading.ac.uk/weather-and-climate-at-reading/files/2020/02/RLee-Fig-1.png</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
<p>
</p>
</div>

# Etape 7 : étude des occurrences saisonnières des régimes

In [ ]:
# Find indexes for end of season
time_str = [x for x in range(len(dates2))]
date_str = [x for x in range(len(dates2))]
for i in range(len(dates2)):
    time_str[i] = str(dates2[i])
    date_str[i] = time_str[i][5:10]

if season_name == 'winter':
    index_end = [i for i, value in enumerate(date_str) if value == '03-31']

if season_name == 'summer':
    index_end = [i for i, value in enumerate(date_str) if value == '08-31']
    
print(index_end)
print(len(index_end))

seasons= [x for x in range(int(season1),int(season2)+1)]
cl1_count=np.zeros(len(index_end))
cl2_count=np.zeros(len(index_end))
cl3_count=np.zeros(len(index_end))
cl4_count=np.zeros(len(index_end))

# Cluster1
cl1_count[0]=list(cluster[0:index_end[0]+1]).count(0)
for i in range(1,len(index_end)):
 cl1_count[i]=list(cluster[index_end[i-1]+1:index_end[i]+1]).count(0)
 
# Cluster2
cl2_count[0]=list(cluster[0:index_end[0]+1]).count(1)
for i in range(1,len(index_end)):
 cl2_count[i]=list(cluster[index_end[i-1]+1:index_end[i]+1]).count(1)

# Cluster3
cl3_count[0]=list(cluster[0:index_end[0]+1]).count(2)
for i in range(1,len(index_end)):
 cl3_count[i]=list(cluster[index_end[i-1]+1:index_end[i]+1]).count(2)

# Cluster4
cl4_count[0]=list(cluster[0:index_end[0]+1]).count(3)
for i in range(1,len(index_end)):
 cl4_count[i]=list(cluster[index_end[i-1]+1:index_end[i]+1]).count(3)

print(seasons)
print(cl1_count)

In [ ]:
# calculate trends
X = np.reshape(seasons, (len(seasons), 1))

y = cl1_count
model = LinearRegression()
model.fit(X, cl1_count)
trend_cl1 = model.predict(X)

y = cl2_count
model = LinearRegression()
model.fit(X, cl2_count)
trend_cl2 = model.predict(X)

y = cl3_count
model = LinearRegression()
model.fit(X, cl3_count)
trend_cl3 = model.predict(X)

y = cl4_count
model = LinearRegression()
model.fit(X, cl4_count)
trend_cl4 = model.predict(X)

def plot_regimes_occ(ax):
 plt.xlabel('Year')
 plt.ylabel('Number of days')
 plt.xlim(int(season1), int(season2))
 plt.ylim(0, 85)
 plt.axvline(1960, color='grey', linestyle='--')
 plt.axvline(1970, color='grey', linestyle='--')
 plt.axvline(1980, color='grey', linestyle='--')
 plt.axvline(1990, color='grey', linestyle='--')
 plt.axvline(2000, color='grey', linestyle='--')
 plt.axvline(2010, color='grey', linestyle='--')
 return ax

fig = plt.figure(figsize=(15, 15))
fig.suptitle('Weather regime occurrences : '
             +var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)
ax = fig.add_subplot(2, 2, 1)
plt.title(regime[0]+' (mean = '+str(int(cl1_count.mean()))+'d)', fontsize=10, loc='center', color=couleur[0])
plot_regimes_occ(ax)
colormat=np.where(cl1_count>cl1_count.mean(), 'red','blue')
plt.bar(seasons,cl1_count-cl1_count.mean(), bottom=cl1_count.mean(), color=colormat, linewidth=1)
plt.plot(seasons,trend_cl1, color='k', linewidth=1)

ax = fig.add_subplot(2, 2, 2)
plt.title(regime[1]+' (mean = '+str(int(cl2_count.mean()))+'d)', fontsize=10, loc='center', color=couleur[1])
plot_regimes_occ(ax)
colormat=np.where(cl2_count>cl2_count.mean(), 'red','blue')
plt.bar(seasons,cl2_count-cl2_count.mean(), bottom=cl2_count.mean(), color=colormat, linewidth=1)
plt.plot(seasons,trend_cl2, color='k', linewidth=1)

ax = fig.add_subplot(2, 2, 3)
plt.title(regime[2]+' (mean = '+str(int(cl3_count.mean()))+'d)', fontsize=10, loc='center', color=couleur[2])
plot_regimes_occ(ax)
colormat=np.where(cl3_count>cl3_count.mean(), 'red','blue')
plt.bar(seasons,cl3_count-cl3_count.mean(), bottom=cl3_count.mean(), color=colormat, linewidth=1)
plt.plot(seasons,trend_cl3, color='k', linewidth=1)

ax = fig.add_subplot(2, 2, 4)
plt.title(regime[3]+' (mean = '+str(int(cl4_count.mean()))+'d)', fontsize=10, loc='center', color=couleur[3])
plot_regimes_occ(ax)
colormat=np.where(cl4_count>cl4_count.mean(), 'red','blue')
plt.bar(seasons,cl4_count-cl4_count.mean(), bottom=cl4_count.mean(), color=colormat, linewidth=1)
plt.plot(seasons,trend_cl4, color='k', linewidth=1)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_regimes_occurrence'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(20, 10))
fig.suptitle('Weather regime occurrences : '
             +var_text+' - '+domain+' - '+season+ ' '+season1+'-'+season2, fontsize=16)

ax = fig.add_subplot(111)
plt.bar(seasons, cl1_count, color=couleur[0])
plt.bar(seasons, cl2_count, bottom=cl1_count, color=couleur[1])
plt.bar(seasons, cl3_count, bottom=cl1_count+cl2_count, color=couleur[2])
plt.bar(seasons, cl4_count, bottom=cl1_count+cl2_count+cl3_count, color=couleur[3])
plt.xlabel("Year")
plt.ylabel("Weather regime occurrence")
plt.xlim(seasons[0]-1,seasons[-1]+1)
plt.legend([regime[0], regime[1], regime[2], regime[3]])

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_regimes_occurrence2'
fig.savefig(figname+'.png', bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Quels régimes semblent caractérisés par des tendances linéaires sur la période totale des données ?</p>  
<p><b>2) </b>Pour les régimes avec tendance, identifier 2 hivers particuliers qui s’opposent à cette tendance ou au contraire qui accentuent cette tendance.
<p><b>3) </b>Pour chaque régime, identifier les hivers avec le maximum/minimum d'occurrence.
</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
<p>
</p>
</div>

<div class="alert alert-warning">
<b>Question codage : classement des occurences de régimes </b>
<p>Le tableau seasons contient les saisons de l'étude. Le tableau regime fait la correspondance entre le numéro de régime et le nom du régime. Les tableaux cl1_count, cl2_count, cl3_count et cl4_count correspondent aux nombres d'occurences par saison de chaque régime de temps.

- Exploiter ces tableaux pour classer, pour chaque régime, les saisons par ordre croissant d'occurence de régime.

Indication : les fonctions suivantes pourront s'avérer utiles :
- https://numpy.org/doc/stable/reference/generated/numpy.sort.html
- https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
</p>
</div>

<div class="alert alert-success">
<b>Réponse dans la cellule de code ci-dessous : </b>
<p>
</p>
</div>

# Etape 8 : corrélation régimes/indices océaniques (Nino 3.4, TNA, AMV)

<div class="alert alert-danger">
<b>Etape à faire uniquement pour les régimes d'hiver (code à adapter pour l'été). </b>
</div>

In [ ]:
#nino=np.loadtxt(dir_data+'nina34.1958-2018.data')
nino=np.loadtxt(dir_data+'nino34.anom.1958-2018.data')
tna=np.loadtxt(dir_data+'tna.1958-2018.data')
amo=np.loadtxt(dir_data+'amon.us.1958-2018.data')

dates = pd.date_range(season1, '2019', freq='M')

nino=nino[0:,1:]
nino=nino.flatten()
nino = Series(nino, index=dates)
print(nino)
nino=nino.rolling(window=3, center=False).mean()
nino=nino[nino.index.month == 3]
nino=(nino-nino.mean())/nino.std()
print(nino)

tna=tna[0:,1:]
tna=tna.flatten()
tna = Series(tna, index=dates)
print(tna)
tna=tna.rolling(window=3, center=False).mean()
tna=tna[tna.index.month == 3]
tna=(tna-tna.mean())/tna.std()
print(tna)

amo=amo[0:,1:]
amo=amo.flatten()
amo = Series(amo, index=dates)
print(amo)
amo=amo.rolling(window=3, center=False).mean()
amo=amo[amo.index.month == 3]
amo=(amo-amo.mean())/amo.std()
print(amo)

years=np.arange(int(season1),2019)

In [ ]:
def plot_ocean(ax):
    plt.title('Data : https://www.esrl.noaa.gov/psd/data/climateindices/list/' , fontsize=12, color='grey')
    plt.xlabel('Year')
    plt.ylabel('SST anomaly (K)')
    plt.xlim(int(season1), 2018)
    plt.axvline(1960, color='grey', linestyle='--', linewidth=0.5)
    plt.axvline(1970, color='grey', linestyle='--', linewidth=0.5)
    plt.axvline(1980, color='grey', linestyle='--', linewidth=0.5)
    plt.axvline(1990, color='grey', linestyle='--', linewidth=0.5)
    plt.axvline(2000, color='grey', linestyle='--', linewidth=0.5)
    plt.axvline(2010, color='grey', linestyle='--', linewidth=0.5)

    plt.axhline(0, color='k')
    plt.axhline(1, color='grey', linestyle='--', linewidth=0.5)
    plt.axhline(1.5, color='grey', linestyle='--', linewidth=0.5)
    plt.axhline(2, color='grey', linestyle='--', linewidth=0.5)
    plt.axhline(-1, color='grey', linestyle='--', linewidth=0.5)
    plt.axhline(-1.5, color='grey', linestyle='--', linewidth=0.5)
    plt.axhline(-2, color='grey', linestyle='--', linewidth=0.5)
    return ax

In [ ]:
fig=plt.figure(figsize=(17, 10))
fig.suptitle('AMO SST anomaly - JFM mean', fontsize=16)
plot_ocean(ax)
plt.plot(years, amo, color='black', linewidth=2)
plt.fill_between(years, amo, where=amo > 0, facecolor='red', interpolate=True) 
plt.fill_between(years, amo, where=amo < 0, facecolor='blue', interpolate=True) 
plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_AMO'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig=plt.figure(figsize=(17, 10))
fig.suptitle('TNA SST anomaly - JFM mean', fontsize=16)
plot_ocean(ax)
plt.plot(years, tna, color='black', linewidth=2)
plt.fill_between(years, tna, where=tna > 0, facecolor='red', interpolate=True) 
plt.fill_between(years, tna, where=tna < 0, facecolor='blue', interpolate=True) 

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_TNA'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig=plt.figure(figsize=(17, 10))
fig.suptitle('Nino 3.4 SST anomaly - JFM mean', fontsize=16)
plot_ocean(ax)
plt.plot(years, nino, color='black', linewidth=2)
plt.fill_between(years, nino, where=nino > 0, facecolor='red', interpolate=True) 
plt.fill_between(years, nino, where=nino < 0, facecolor='blue', interpolate=True)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_Nino'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig=plt.figure(figsize=(17, 10))
fig.suptitle('Oceanic indices - Annual mean', fontsize=16)
plt.title('Data : https://www.esrl.noaa.gov/psd/data/climateindices/list/' , fontsize=12, color='grey')
plt.xlabel('Year')
plt.ylabel('Oceanic index')
ax = nino.plot(color='red', linewidth=2, label='Niño 3.4 SST anomaly - JFM mean')
ax = tna.plot(color='blue', linewidth=2, label='TNA SST anomaly - JFM mean')
ax = amo.plot(color='green', linewidth=2, label='AMO SST anomaly - JFM mean')

plt.axhline(0, color='k')
plt.axhline(1, color='grey', linestyle='--', linewidth=0.5)
plt.axhline(1.5, color='grey', linestyle='--', linewidth=0.5)
plt.axhline(2, color='grey', linestyle='--', linewidth=0.5)
plt.axhline(-1, color='grey', linestyle='--', linewidth=0.5)
plt.axhline(-1.5, color='grey', linestyle='--', linewidth=0.5)
plt.axhline(-2, color='grey', linestyle='--', linewidth=0.5)

plt.legend()
plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_cor_regimes_ocean'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
cl1_count_a=((cl1_count-cl1_count.mean())/cl1_count.mean())*100
cl2_count_a=((cl2_count-cl2_count.mean())/cl2_count.mean())*100
cl3_count_a=((cl3_count-cl3_count.mean())/cl3_count.mean())*100
cl4_count_a=((cl4_count-cl4_count.mean())/cl4_count.mean())*100

print(nino.shape)
print(cl1_count_a.shape) # 2 dernières années en +
print(cl1_count_a[0:-2].shape)

In [ ]:
cor_nino1=np.corrcoef(nino, cl1_count_a[0:-2])
cor_nino2=np.corrcoef(nino, cl2_count_a[0:-2])
cor_nino3=np.corrcoef(nino, cl3_count_a[0:-2])
cor_nino4=np.corrcoef(nino, cl4_count_a[0:-2])

print("NINO34 correlation")
print(regime[0]+" : ", cor_nino1[1,0])
print(regime[1]+" : ", cor_nino2[1,0])
print(regime[2]+" : ", cor_nino3[1,0])
print(regime[3]+" : ", cor_nino4[1,0])

cor_tna1=np.corrcoef(tna, cl1_count_a[0:-2])
cor_tna2=np.corrcoef(tna, cl2_count_a[0:-2])
cor_tna3=np.corrcoef(tna, cl3_count_a[0:-2])
cor_tna4=np.corrcoef(tna, cl4_count_a[0:-2])
print("TNA correlation")
print(regime[0]+" : ", cor_tna1[1,0])
print(regime[1]+" : ", cor_tna2[1,0])
print(regime[2]+" : ", cor_tna3[1,0])
print(regime[3]+" : ", cor_tna4[1,0])

cor_amo1=np.corrcoef(amo, cl1_count_a[0:-2])
cor_amo2=np.corrcoef(amo, cl2_count_a[0:-2])
cor_amo3=np.corrcoef(amo, cl3_count_a[0:-2])
cor_amo4=np.corrcoef(amo, cl4_count_a[0:-2])
print("AMO correlation")
print(regime[0]+" : ", cor_amo1[1,0])
print(regime[1]+" : ", cor_amo2[1,0])
print(regime[2]+" : ", cor_amo3[1,0])
print(regime[3]+" : ", cor_amo4[1,0])

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Quelles sont les corrélations les plus fortes entre régimes de temps et indices océaniques en hiver ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
<p>
</p>
</div>

# Etape 8 : retour sur une saison particulière

In [ ]:
if season_name == 'winter':
    choix=input('hiver (pour un hiver N-N+1, entrer N) : ')
    leap_year=calendar.isleap(int(choix)+1)
    if var_text=='Z500':
        date1=choix+'-12-01T18'
    if var_text=='MSLP':
        date1=choix+'-12-01T09'
    idx_date1=dates2.get_loc(date1, method ='ffill')
    if leap_year:
        print('Année bisextile')
        idx_date2=idx_date1+122
    else:
        idx_date2=idx_date1+121
    
if season_name == 'summer':
    choix=input('été ? ')
    if var_text=='Z500':
        date1=choix+'-06-01T18'
    if var_text=='MSLP':
        date1=choix+'-06-01T09'
    idx_date1=dates2.get_loc(date1, method ='ffill')
    idx_date2=idx_date1+121

In [ ]:
time=dates2[idx_date1:idx_date2]
print(time.shape)

date1=str(time[0])[0:10]
date2=str(time[-1])[0:10]

time_str = [x for x in range(len(time))]
date_str = [x for x in range(len(time))]
date_str_title = [x for x in range(len(time))]

for i in range(len(time)):
    time_str[i] = str(time[i])
    date_str_title[i] = time_str[i][0:10]	
    date_str[i] = time_str[i][5:10]
    
leap_year=calendar.isleap(int(time_str[-1][0:4]))
if leap_year:
    print('Année bisextile')

In [ ]:
data_month    = data_season.sel(time=slice(date1,date2))
z=data_month[varname]/var_div

data_month_anom    = data_anom.sel(time=slice(date1,date2))
z_anom=data_month_anom[varname]/var_div

lat  = data_month_anom.lat.values
lon  = data_month_anom.lon.values

In [ ]:
if varname=='z':
    plt_title2 = 'Geopotential height anomaly ('+units+') at 500 hPa : '+ str(date1)+'-'+str(date2)

if varname=='msl':
    plt_title2 = 'Mean Sea Level Pressure anomaly ('+units+') : '+ str(date1)+'-'+str(date2)

In [ ]:
axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(17,10))
fig.suptitle(plt_title2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(5, 6),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    reg=cluster[idx_date1+i] # régime
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_title(date_str_title[i]+' '+regime[reg], fontsize=10, color=colors[idx_date1+i])
    p1 = ax.contourf(lon, lat, z_anom[i,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname=dir_figs+varname+'_anom_'+domain_name+'_'+season_name+'_cluster_'+str(date1)+'-'+str(date2)+'_1'
fig.savefig(figname+'.png', bbox_inches='tight')

#
axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(17,10))
fig.suptitle(plt_title2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(5, 6),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

for i, ax in enumerate(axgr):
    reg=cluster[idx_date1+30+i] # régime
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_title(date_str_title[i+30]+' '+regime[reg], fontsize=10, color=colors[idx_date1+i+30])
    p1 = ax.contourf(lon, lat, z_anom[i+30,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i+30,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname=dir_figs+varname+'_anom_'+domain_name+'_'+season_name+'_cluster_'+str(date1)+'-'+str(date2)+'_2'
fig.savefig(figname+'.png', bbox_inches='tight')

#
axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(17,10))
fig.suptitle(plt_title2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(5, 6),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

for i, ax in enumerate(axgr):
    reg=cluster[idx_date1+60+i] # régime
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_title(date_str_title[i+60]+' '+regime[reg], fontsize=10, color=colors[idx_date1+i+60])
    p1 = ax.contourf(lon, lat, z_anom[i+60,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i+60,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname=dir_figs+varname+'_anom_'+domain_name+'_'+season_name+'_cluster_'+str(date1)+'-'+str(date2)+'_3'
fig.savefig(figname+'.png', bbox_inches='tight')

#
axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(17,10))
fig.suptitle(plt_title2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(5, 6),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

for i, ax in enumerate(axgr):
    reg=cluster[idx_date1+90+i] # régime
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_title(date_str_title[i+90]+' '+regime[reg], fontsize=10, color=colors[idx_date1+i+90])
    p1 = ax.contourf(lon, lat, z_anom[i+90,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i+90,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname=dir_figs+varname+'_anom_'+domain_name+'_'+season_name+'_cluster_'+str(date1)+'-'+str(date2)+'_4'
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
# Nombre de jours par régime sur la saison choisie.
nc1s=list(cluster[idx_date1:idx_date2]).count(0)
nc2s=list(cluster[idx_date1:idx_date2]).count(1)
nc3s=list(cluster[idx_date1:idx_date2]).count(2)
nc4s=list(cluster[idx_date1:idx_date2]).count(3)

# Fonction pour la légende.
def plot_regimes_leg(ax):
    patch1 = mpatches.Patch(color=couleur[0], label=regime[0]+' : '+str(nc1s)+' ('+str(f1s)+'%)')
    patch2 = mpatches.Patch(color=couleur[1], label=regime[1]+' : '+str(nc2s)+' ('+str(f2s)+'%)')
    patch3 = mpatches.Patch(color=couleur[2], label=regime[2]+' : '+str(nc3s)+' ('+str(f3s)+'%)')
    patch4 = mpatches.Patch(color=couleur[3], label=regime[3]+' : '+str(nc4s)+' ('+str(f4s)+'%)')
    all_handles = (patch1, patch2, patch3, patch4)
    leg = ax.legend(handles=all_handles, loc='lower right')
    return ax

f1s=round(nc1s/cluster[idx_date1:idx_date2].shape[0]*100,2)
f2s=round(nc2s/cluster[idx_date1:idx_date2].shape[0]*100,2)
f3s=round(nc3s/cluster[idx_date1:idx_date2].shape[0]*100,2)
f4s=round(nc4s/cluster[idx_date1:idx_date2].shape[0]*100,2)

fig = plt.figure(figsize=(15, 2))
fig.suptitle('Weather regimes : '+date_str_title[0]+' to '+date_str_title[-1], fontsize=14)

ax = fig.add_subplot(1, 1, 1)

# create a collection with a rectangle for each day
col = PatchCollection([Rectangle((y, 0), 1, 1) for y in range(len(time))])

#data = cluster[idx_date1:idx_date2]
#col.set_array(data)
col.set_color(colors[idx_date1:idx_date2])
ax.add_collection(col)

if season_name == 'winter':
    labels = ("Dec", "Jan", "Feb", "Mar", "Avr")
    positions = (0, 31, 31+31, 31+31+28, 31+31+28+31)
    if leap_year:
        positions = (0, 31, 31+31, 31+31+29, 31+31+29+31)

if season_name == 'summer':
    positions = (0, 30, 30+31, 30+31+31, 30+31+31+30)
    labels = ("Jun", "Jul", "Aug", "Sep", "Oct")

plt.xticks(positions, labels)
ax.set_ylim(0, 1)
ax.set_xlim(0, len(time))
ax.yaxis.set_major_formatter(plt.NullFormatter())
plt.axvline(x=positions[0], c='black')
plt.axvline(x=positions[1], c='black')
plt.axvline(x=positions[2], c='black')
plt.axvline(x=positions[3], c='black')

plot_regimes_leg(ax)

fig.savefig(dir_figs+varname+'_'+domain_name+'_'+season_name+
            '_regimes-stripes_'+date_str_title[0]+'-'+date_str_title[-1]+'.png', bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Quel a été le régime de temps dominant au cours de l'hiver 2019-2020 ? Principalement au cours de quels mois ?</p>
<br>
<b>Relancer l'étape 8 pour l'hiver 2011-2012 : </b>
<p><b>2) </b>Quel régime de temps a été absent au cours de l'hiver 2011-2012 ?</p>
<p><b>3) </b>Quel régime de temps a dominé en début de mois de février 2012 ? Quelle en a été la conséquence sur le temps sensible en France ?</p>
<br>
<b>Relancer l'étape 8 pour l'hiver 2009-2010 : </b>
<p><b>4) </b>Quel a été le régime de temps dominant au cours de l'hiver 2009-2010 ?  Au cours de quel mois les occurrences de ce régime ont-elles été les plus nombreuses ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
<p>
</p>
</div>

# Autres visualisations

In [ ]:
for i in range(len(time)):
    #print(date_str_title[i])
    reg=cluster[idx_date1+i]
    if reg==0:
            composite  = mean_c1_anom
            composite_mean = mean_c1
    if reg==1:
            composite  = mean_c2_anom
            composite_mean = mean_c2
    if reg==2:
            composite  = mean_c3_anom
            composite_mean = mean_c3
    if reg==3:
            composite  = mean_c4_anom
            composite_mean = mean_c4
            
    fig = plt.figure(figsize=(15., 5.))
    fig.suptitle(plt_title2, fontsize=16)
    
    ax = fig.add_subplot(1,2,1, projection=projection)
    ax.set_title(date_str_title[i]+' '+regime[reg], fontsize=10, color=colors[idx_date1+i])
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    p1 = ax.contourf(lon, lat, z_anom[i,:,:], levels2, transform=ccrs.PlateCarree(), cmap=cmap2, extend='both')
    p2 = ax.contour(lon, lat, z_anom[i,:,:], levels2, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    p3 = ax.contour(lon, lat, z[i,:,:], levels1, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(p3, inline=1, fmt='%4.0f', fontsize=10)
    cb = fig.colorbar(p1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('Anomaly ('+units+')', fontsize=12)
    
    ax = fig.add_subplot(1,2,2, projection=projection)
    plt.title(regime[reg]+' composite', fontsize=10, loc='center', color=colors[idx_date1+i])
    ax.coastlines()
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    cf = ax.contourf(lons, lats, composite, levels=clevs_anom, 
                     cmap=cmap2, extend='both', transform=ccrs.PlateCarree())
    c0 = ax.contour(lons, lats, composite, levels=clevs_anom, colors='black', linewidths=0.2,
                    transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, composite_mean, levels=clevs, colors='black', linewidths=1,
                   transform=ccrs.PlateCarree())
    ax.clabel(c, inline=1, fmt='%4.0f', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('Anomaly ('+units+')', fontsize=12)

    figname=dir_anim+varname+'_anom_cluster'+domain_name+'_'+season_name+'_'+date_str_title[i]
    fig.savefig(figname+'.png', bbox_inches='tight')
    plt.close()
print('png files are in the animation folder, ready to make the animation')

In [ ]:
gif_filepath = dir_anim+varname+'_anom_cluster_'+domain_name+'_'+season_name+'_'+str(date1)+'-'+str(date2)+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.suptitle('PC1 and PC2 phase space : '+date_str_title[0]+' to '+date_str_title[-1], fontsize=14)

ax = fig.add_subplot(1, 1, 1)
plot_phase_space2d(ax)
ax.scatter(pcs[idx_date1:idx_date2,0], pcs[idx_date1:idx_date2,1], c=colors[idx_date1:idx_date2], s=10)
ax.scatter(centroids[:, 0], centroids[:, 1], c=couleur, s=2000, alpha=1, marker='+')

for i,type in enumerate(date_str):
    x = pc1[idx_date1+i]
    y = pc2[idx_date1+i]
    plt.text(x, y, type, fontsize=6)

plot_regimes_leg(ax)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pc12_'+date_str_title[0]+'-'+date_str_title[-1]
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.suptitle('PC phase space : '+date_str_title[0]+' to '+date_str_title[-1], fontsize=14)

ax = fig.add_subplot(111, projection='3d')
plot_phase_space3d(ax)
ax.scatter(pcs[idx_date1:idx_date2,0], pcs[idx_date1:idx_date2,1], pcs[idx_date1:idx_date2,2],
           c=colors[idx_date1:idx_date2], s=10)
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], c=couleur, s=200, alpha=1, marker='+')

plot_regimes_leg(ax)

plt.show()

figname=dir_figs+varname+'_'+domain_name+'_'+season_name+'_pc123_'+date_str_title[0]+'-'+date_str_title[-1]
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
# loop for each day	
for i in range(len(time)):
 #print(date_str[i])
 fig = plt.figure(figsize=(10, 10))
 fig.suptitle('PC1 and PC2 phase space : '+date_str_title[0]+' to '+date_str_title[-1], fontsize=14)
 ax = fig.add_subplot(1, 1, 1)
 plot_phase_space2d(ax)
 ax.scatter(pcs[idx_date1:idx_date1+i+1,0], pcs[idx_date1:idx_date1+i+1,1], 
            c=colors[idx_date1:idx_date1+i+1], s=10)
 ax.scatter(centroids[:, 0], centroids[:, 1], c=couleur, s=2000, alpha=1, marker='+')
 x = pc1[idx_date1+i]
 y = pc2[idx_date1+i]
 plt.text(x, y, date_str[i], fontsize=10)
 plot_regimes_leg(ax)
 figname=dir_anim+varname+'_'+domain_name+'_'+season_name+'_PC_2d_phase_space_4clusters_'+date_str_title[i]
 fig.savefig(figname+'.png')
 plt.close()

In [ ]:
gif_filepath = dir_anim+varname+'_'+domain_name+'_'+season_name+'_PC_2d_phase_space_4clusters_'+date_str_title[0]+'-'+date_str_title[-1]+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
import scipy.spatial.distance as sdist
dist=sdist.cdist(pcs, centroids)
dist_c1=dist[:,0]
dist_c2=dist[:,1]
dist_c3=dist[:,2]
dist_c4=dist[:,3]

def plot_dist(ax):
    #plt.ylim(0, 5)
    plt.axhline(0, color='k')
    plt.xlim(xmin=datetime.datetime(int(str(time[0])[0:4]), int(str(time[0])[5:7]),
                                    int(str(time[0])[8:10])),
             xmax=datetime.datetime(int(str(time[-1])[0:4]), int(str(time[-1])[5:7]),
                                    int(str(time[-1])[8:10])))
    #ax.spines['right'].set_visible(False)
    #ax.spines['top'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    #ax.axes.get_yaxis().set_visible(False)
    return ax

d1=dist_c1[idx_date1:idx_date2] # distance au régime 1
d2=dist_c2[idx_date1:idx_date2] # distance au régime 2
d3=dist_c3[idx_date1:idx_date2] # distance au régime 3
d4=dist_c4[idx_date1:idx_date2] # distance au régime 4

fig=plt.figure(figsize=(20, 7))
fig.suptitle('Regime distance from centroid', fontsize=16)

ax = fig.add_subplot(4, 1, 1)
plot_dist(ax)
plt.scatter(time, d1, color='blue', s=200000/d1, label='Distance '+regime[0])
plt.legend()

ax = fig.add_subplot(4, 1, 2)
plot_dist(ax)
plt.scatter(time, d2, color='red', s=200000/d2, label='Distance '+regime[1])
plt.legend()

ax = fig.add_subplot(4, 1, 3)
plot_dist(ax)
plt.scatter(time, d3, color='green', s=200000/d3, label='Distance '+regime[2])
plt.legend()

ax = fig.add_subplot(4, 1, 4)
plot_dist(ax)
plt.scatter(time, d4, color='orange', s=200000/d4, label='Distance '+regime[3])
plt.axhline(0, color='k')
plt.legend()

plt.show()

fig.savefig(dir_figs+varname+'_'+domain_name+'_'+season_name+
            '_regimes-distances_'+date_str_title[0]+'-'+date_str_title[-1]+'.png', bbox_inches='tight')

# Bonus : les frises des régimes de tous les hivers de la base de données

<div class="alert alert-danger">
<b>Etape à faire uniquement pour les régimes d'hiver (code à adapter pour l'été). </b>

<br>
<br>

A comparer avec le produit de Simon Lee :
    
https://simonleewx.files.wordpress.com/2022/10/atlantic_regimes_timeseries_by_winter_1980-2022.png
</div>

In [ ]:
w=np.arange(1957,2020,1)
winters = [str(a) for a in w]

for winter in winters:
    leap_year=calendar.isleap(int(winter)+1)
    if var_text=='Z500':
        date1=winter+'-12-01T18'
    if var_text=='MSLP':
        date1=winter+'-12-01T09'
    idx_date1=dates2.get_loc(date1, method ='ffill')
    if leap_year:
        idx_date2=idx_date1+122
    else:
        idx_date2=idx_date1+121
    
    time=dates2[idx_date1:idx_date2]

    date1=str(time[0])[0:10]
    date2=str(time[-1])[0:10]

    time_str = [x for x in range(len(time))]
    date_str = [x for x in range(len(time))]
    date_str_title = [x for x in range(len(time))]

    for i in range(len(time)):
        time_str[i] = str(time[i])
        date_str_title[i] = time_str[i][0:10]	
        date_str[i] = time_str[i][5:10]

    leap_year=calendar.isleap(int(time_str[-1][0:4]))

    data_month    = data_season.sel(time=slice(date1,date2))
    z=data_month[varname]/var_div

    data_month_anom    = data_anom.sel(time=slice(date1,date2))
    z_anom=data_month_anom[varname]/var_div

    lat  = data_month_anom.lat.values
    lon  = data_month_anom.lon.values

    if varname=='z':
        plt_title2 = 'Geopotential height anomaly ('+units+') at 500 hPa : '+ str(date1)+'-'+str(date2)

    if varname=='msl':
        plt_title2 = 'Mean Sea Level Pressure anomaly ('+units+') : '+ str(date1)+'-'+str(date2)

    nc1s=list(cluster[idx_date1:idx_date2]).count(0)
    nc2s=list(cluster[idx_date1:idx_date2]).count(1)
    nc3s=list(cluster[idx_date1:idx_date2]).count(2)
    nc4s=list(cluster[idx_date1:idx_date2]).count(3)

    f1s=round(nc1s/cluster[idx_date1:idx_date2].shape[0]*100,2)
    f2s=round(nc2s/cluster[idx_date1:idx_date2].shape[0]*100,2)
    f3s=round(nc3s/cluster[idx_date1:idx_date2].shape[0]*100,2)
    f4s=round(nc4s/cluster[idx_date1:idx_date2].shape[0]*100,2)

    fig = plt.figure(figsize=(15, 2))
    fig.suptitle('Weather regimes : '+date_str_title[0]+' to '+date_str_title[-1], fontsize=14)

    ax = fig.add_subplot(1, 1, 1)

    # create a collection with a rectangle for each day
    col = PatchCollection([Rectangle((y, 0), 1, 1) for y in range(len(time))])

    #data = cluster[idx_date1:idx_date2]
    #col.set_array(data)
    col.set_color(colors[idx_date1:idx_date2])
    ax.add_collection(col)
    
    if season_name == 'winter':
        labels = ("Dec", "Jan", "Feb", "Mar", "Avr")
        positions = (0, 31, 31+31, 31+31+28, 31+31+28+31)
        if leap_year:
            positions = (0, 31, 31+31, 31+31+29, 31+31+29+31)

    if season_name == 'summer':
        positions = (0, 30, 30+31, 30+31+31, 30+31+31+30)
        labels = ("Jun", "Jul", "Aug", "Sep", "Oct")

    plt.xticks(positions, labels)
    ax.set_ylim(0, 1)
    ax.set_xlim(0, len(time))
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    plt.axvline(x=positions[0], c='black')
    plt.axvline(x=positions[1], c='black')
    plt.axvline(x=positions[2], c='black')
    plt.axvline(x=positions[3], c='black')
    plt.axvline(x=positions[4], c='black')
    
    plot_regimes_leg(ax)
    
    plt.show()

    fig.savefig(dir_figs+varname+'_'+domain_name+'_'+season_name+
                '_regimes-stripes_'+date_str_title[0]+'-'+date_str_title[-1]+'.png', bbox_inches='tight')

<div class="alert alert-warning">
<b>Tâches supplémentaires : </b>
<p><b>1) </b>Consulter les produits de suivi des régimes de temps de l'équipe prévisions saisonnières de Météo-France : 

http://seasonal.meteo.fr/content/suivi-clim-regimes-quot
    
http://seasonal.meteo.fr/content/suivi-clim-regimes-trim

</p>
<p><b>2) </b>Consulter les prévisions à long terme (J+46) des régimes de temps Atlantique réalisées par le centre Européen de prévisions météorologiques ECMWF :
    
https://charts.ecmwf.int/products/extended-regime-probabilities
</p>
<p><b>3) </b> Refaire le calepin afin d’extraire les régimes de Pmer sur l’Atlantique Nord pour les mois d’été (juin-juillet-août-septembre).</p>
</div>

<div class="alert alert-danger">
<b>Défi codage n°1 (*) : séquences record de régimes </b>
<p>
Le tableau cluster contient les numéros de régime pour chaque jour de l'étude (0 = régime 1, 1 = régime 2, 2 = régime 3, 3 = régime 4). Le tableau regime fait la correspondance entre le numéro de régime et le nom du régime. L'index de date dates2 contient les dates de l'étude.

- Exploiter ces 3 tableaux pour établir le top 20 des séquences les plus longues d'un même régime en précisant pour chaque séquence le régime considéré ainsi que la date de de début et de fin de la séquence.

Indication : les fonctions suivantes pourront s'avérer utiles :
- https://docs.python.org/3/library/itertools.html#itertools.groupby
- https://numpy.org/doc/stable/reference/generated/numpy.sort.html
- https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
</p>
</div>

<div class="alert alert-danger">
<b>Défi codage N°2 (**) : non prise en compte des régimes non persistants : </b>
<p>
Le tableau cluster contient les numéros de régime pour chaque jour de l'étude (0 = régime 1, 1 = régime 2, 2 = régime 3, 3 = régime 4).

- Exploiter ce tableau pour déclassifier les séquences de moins de 3 jours consécutifs d'un même régime en les attribuant à un nouveau numéro de régime (4 = classe régime "poubelle"). Ces séqeunces correspondent en effet à des phases de transitions entre 2 régimes.

Indication : la fonction suivante pourra s'avérer utile :
- https://docs.python.org/3/library/itertools.html#itertools.groupby
</p>
</div>

<div class="alert alert-danger">
<b>Défi codage N°3 (**) : non prise en compte des régimes non robustes : </b>
<p>
Le tableau cluster contient les numéros de régime pour chaque jour de l'étude (0 = régime 1, 1 = régime 2, 2 = régime 3, 3 = régime 4). Les dataArray mean_c1_anom, mean_c2_anom, mean_c3_anom, mean_c3_anom correspondent aux composites des régimes 1 à 4. Le dataArray data_anom[varname][:,:,:]/var_div contient les champs d'anomalies sur toute la période d'étude.

- Exploiter ces données pour déclassifier les régimes dont la corrélation spatiale avec le composite est inférieure à 0.25 en les attribuant à un régime 4 (classe régime "poubelle"). Ces séquences correspondent en effet à des régimes non robustes.

Indication : les fonctions suivantes pourront s'avérer utiles :
- https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html
- https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html (pour aplatir les tableaux 2D)
</p>
</div>